# Constraint satisfaction problem

In the previous tasks the state was opaque: an algorithm could execute an action and transform a state to another state, but that was it. It could not depend in its search on the details of a state, because the state representation was different from problem to problem. In CSP this is no longer true: the state always consists of an assignment of values over some domains to variables. This enables an algorithm to interact with the state directly, not only through opaque actions. 

Allowed assignments are limited by constraints in the problem and broadly speaking they can be divided into three groups:

* *Unary constraints* each concerning only a single variable (e.g., $x>0$)
* *Binary constraints* relating two variables to each other (e.g., $x>y$)
* *General constraints* concerning more than two variables at once

In general, the shape of constraints is unrestricted and we are not limited to numeric domains and constraints in the form of inequalities. In principle, one could employ arbitrary boolean functions as constraints.

The following class `CSP` is a very limited representation of such a problem capable only of handling:

* finite domains that can be materialized (e.g., the set of all 32-bit integers is doable in theory; the set of all 64-bit integers is out of reach) 
* unary constraints by setting the domain of a variable appropriately;
* binary constraints defined in terms of Python's operators `==`, `!=`, `<`, `<=`, `>` and `>=` such that both sides are variables.

About its methods:

* The constructor (`__init__`) requires no arguments.
* `addVariable` requires two arguments: 
    * `var` - a variable (a string representing its name, but really, it could be any object);
    * `domain` - a collection of the allowed values for the variable
* `addBinaryConstraint` also requires three arguments:
    * `var1` - a variable, the left-hand side of the constraint
    * `op` - an operator (one of the following strings: `'=='`, `'!='`, `'<'`, `'<='`, `'>'`, `'>='`)
    * `var2` - a variable, the right-hand side of the constraint
* `verify` is used to verify whether the relation given by the second argument `op` holds between the values given as the first argument `left` and the third argument `right`. Returns `True` if it does and `False` otherwise.
* `is_complete` gets a single argument: a dictionary representing an assignment of values to variables. From now on, we call such a dictionary an *assignment*. It returns `True` if all the variables of the problem are present in the dictionary, and `False` otherwise.
* `is_consistent` also requries an assignment. It returns `True` if all the values in the assignment are such that they come from the respecitve domains, and no binary constrain is violated (but if the assignment is not complete then some may be unfulfilled).

In [1]:
class CSP:
    def __init__(self):
        self.domains = {}
        self.binary = []
    
    def addVariable(self, var, domain):        
        assert var not in self.domains
        self.domains[var] = set(domain)
        
    def addBinaryConstraint(self, var1, operator, var2):
        assert var1 in self.domains
        assert var2 in self.domains
        c = (var1, operator, var2)
        self.binary.append(c)      
        
    def verify(self, left, op, right):
        if op[0] == '=':
            return left == right
        if op == '!=':
            return left != right
        if op == '<':
            return left < right
        if op == '<=':
            return left <= right
        if op == '>':
            return left > right
        if op == '>=':
            return left >= right
        
    def is_complete(self, assignment):
        return self.domains.keys() <= assignment.keys() 
        
    def is_consistent(self, assignment):
        for var, value in assignment.items():            
            if value not in self.domains[var]:
                return False
        for var1, op, var2 in self.binary:
            if var1 in assignment and var2 in assignment:
                if not self.verify(assignment[var1], op, assignment[var2]):
                    return False
        return True

## Coloring Australia

Lets model a map coloring problem. In particular, we'll color the following map of Australia. It can be simplified to a graph presented in the right-hand side of the picture below.

![](aima-images/fig_6_1.png)

We begin by creating a new problem in the variable `australia` over the domain consisting of three values: `'R'` statnding for the color red, `'G'` standing for green and `'B'` standing for blue. Of course the particular symbols used are not relevant. There are 7 teritories, so we add 7 variables to the problem.

In [2]:
australia = CSP()
australia.addVariable('WA', {'R', 'G', 'B'})
australia.addVariable('NT', {'R', 'G', 'B'})
australia.addVariable('SA', {'R', 'G', 'B'})
australia.addVariable('Q', {'R', 'G', 'B'})
australia.addVariable('NSW', {'R', 'G', 'B'})
australia.addVariable('V', {'R', 'G', 'B'})
australia.addVariable('T', {'R', 'G', 'B'})

Any two teritories of Australia sharing a border should be colored using different colours. We thus define 9 binary constraints.

In [3]:
australia.addBinaryConstraint('WA', '!=', 'NT')
australia.addBinaryConstraint('WA', '!=', 'SA')
australia.addBinaryConstraint('NT', '!=', 'SA')
australia.addBinaryConstraint('NT', '!=', 'Q')
australia.addBinaryConstraint('SA', '!=', 'Q')
australia.addBinaryConstraint('SA', '!=', 'NSW')
australia.addBinaryConstraint('SA', '!=', 'V')
australia.addBinaryConstraint('Q', '!=', 'NSW')
australia.addBinaryConstraint('NSW', '!=', 'V')

Let's see how the `verify` method works.

In [4]:
print("Is G=G?", australia.verify('G', '==', 'G'))
print("Is 5<7?", australia.verify(5, '<', 7))
print("Is 5>=7?", australia.verify(5, '>=', 7))

Is G=G? True
Is 5<7? True
Is 5>=7? False


Lets now consider an assignment such that *Western Australia* is red and the rest is unspecified. Is it a consistent assignment?

In [5]:
australia.is_consistent({'WA': 'R'})

True

Is it a complete assignment?

In [6]:
australia.is_complete({'WA': 'R'})

False

As expected it didn't work: there are still 9 unfulfiled constraints. So lets extend the solution.

In [7]:
assignment = {'WA': 'R', 'Q': 'R', 'V': 'R', 'NT': 'B', 'NSW': 'B', 'SA': 'G'}
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))

Is consistent? True
Is complete? False


Now it almost works: all the constraints are satisfied, but the solution is still not complete, as Tasmania is still colorless.

In [8]:
assignment = {'WA': 'R', 'Q': 'R', 'V': 'R', 'NT': 'B', 'NSW': 'B', 'SA': 'G', 'T': 'B'}
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))

Is consistent? True
Is complete? True


Finally, we have manually collored the map of Australia! Now let's make computer do this for us.

## Task 1: Backtracking

Complete the following cell of code by implementing the recursive backtracking algorithm. Basically, it is a recursive DFS where an action is assigning a value to a variable. Avoid making copies of assignments, instead operate on a single assignment, and update it accordingly. Increment `self.counter` every time you check wheter an assigment is a solution (i.e., is consistent and complete). Use the provided methods `select_unassigned_variable` and `order_domain_values` to, respectively, get a next variable to assign a value to, and to get the list of allowed values for the variable. `solve` should return a solution (i.e., a complete, consistent assignment) or `None`.

In [9]:
class RecursiveBacktracking:
    def __init__(self, csp):
        self.csp = csp
        self.counter = 0
        
    def select_unassigned_variable(self, assignment):
        for var in self.csp.domains.keys():
            if var not in assignment:
                return var
        return None
    
    def order_domain_values(self, variable, assignment):
        return self.csp.domains[variable]
        
    
    def solve(self):
        self.counter = 0
        return self._solve_rec({})

        
    def _solve_rec(self, assignment):
        self.counter += 1
        if not self.csp.is_consistent(assignment):
            return None
        
        if self.csp.is_complete(assignment):
            # all variables are used and we got specific value, so we can freely return
            return assignment

        variable = self.select_unassigned_variable(assignment)
        # shouldn't be reachable
        # if not variable:
        #     return None

        for value in self.order_domain_values(variable, assignment):
            assignment[variable] = value

            solution = self._solve_rec(assignment)

            if solution:
                return solution
            del assignment[variable]
        
        return None

Let's test it on the Australia problem! In the cell below you should get a consistent, complete assignment, i.e., a solution to the problem.

In [10]:
solver = RecursiveBacktracking(australia)
assignment = solver.solve()
print("Assignment", assignment)
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))
print("# considered assignments", solver.counter)

Assignment {'WA': 'R', 'NT': 'G', 'SA': 'B', 'Q': 'R', 'NSW': 'G', 'V': 'R', 'T': 'R'}
Is consistent? True
Is complete? True
# considered assignments 12


## Task 2: Extend `RecursiveBacktracking` with the MRV and degree heuristics

Modify your `RecursiveBacktracking` to include the MRV (minimum remaining values) heuristics and the degree heuristics. Implement them in the appropriate places below in the class `RecursiveBacktrackingWithHeuristics`. If you want, you can also implement the least constraining value heuristics, but this is not mandatory. Follow the same assumptions as for `RecursiveBacktracking`, so expose the main backtracking algorithm as the `solve` and keep track of the number of considered assignments in the `counter` variable.

In [11]:
class RecursiveBacktrackingWithHeuristics:
    def __init__(self, csp: CSP):
        self.csp = csp
        self.counter = 0
        
    def select_unassigned_variable(self, assignment):
        min_sum = None

        available_variables = list(filter(lambda v: v not in assignment, self.csp.domains.keys()))
        consistency_dict = dict()

        # MRV: find out mininum legal remaining value count
        for var in available_variables:
            temp_sum = 0

            for value in self.csp.domains[var]:
                flag_consistent = True

                # verify the subset of constraints
                for var1, op, var2 in self.csp.binary:
                    if var1 == var and var2 in assignment:
                        flag_consistent = self.csp.verify(value, op, assignment[var2])
                    elif var2 == var and var1 in assignment:
                        flag_consistent = self.csp.verify(assignment[var1], op, value)
                    
                    if not flag_consistent:
                        break

                if flag_consistent:
                    temp_sum += 1
                    consistency_dict[var] = consistency_dict.get(var, []) + [value]

            # No legal values left
            if not temp_sum:
                return None

            if not min_sum or min_sum > temp_sum:
                min_sum = temp_sum

        most_constraining_var = None
        most_constraining_count = None

        # Degree: within all variables with minimum remaining values find the most constraining variable

        constraints_with_available_vars = list(filter(
            lambda c: c[0] in available_variables or c[2] in available_variables, 
            self.csp.binary
        ))

        for var in available_variables:
            temp_sum = len(consistency_dict[var])

            if temp_sum != min_sum:
                continue

            # now we count restrictions: we find minimum constraining variable on remaining
            temp_sum = len(list(filter(lambda constraint: var in constraint, constraints_with_available_vars)))

            if not most_constraining_count or most_constraining_count < temp_sum:
                most_constraining_count = temp_sum
                most_constraining_var = var

        return most_constraining_var, consistency_dict[most_constraining_var]
    
    def order_domain_values(self, variable, assignment):
        return self.csp.domains[variable]
        
    def solve(self):
        self.counter = 0
        return self._solve_rec({})
    
    def _solve_rec(self, assignment):
        self.counter += 1
        if self.csp.is_complete(assignment):
            # all variables are used and we got specific value, so we can freely return
            return assignment

        variable, dom = self.select_unassigned_variable(assignment)
        if not variable:
            return None

        for value in dom:
            assignment[variable] = value
            solution = self._solve_rec(assignment)

            if solution:
                return solution
        
        del assignment[variable]
        return None

Lets test your implementation by solving the problem of coloring Australia!

In [12]:
solver = RecursiveBacktrackingWithHeuristics(australia)
assignment = solver.solve()
print("Assignment", assignment)
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))
print("# considered assignments", solver.counter)

Assignment {'SA': 'R', 'NT': 'G', 'Q': 'B', 'NSW': 'G', 'WA': 'B', 'V': 'B', 'T': 'R'}
Is consistent? True
Is complete? True
# considered assignments 8


**Compare the number of considered assignments for `RecursiveBacktracking` and `RecursiveBacktrackingWithHeuristics`. How do they differ? Why do you think is that?**

*Answer*

- `RecursiveBacktracking` is just a brute-force approach. So nothing intelligent. The instance of the problem is quite easy, thus it does not differ to much from the minimum amount of checks.
- `RecursiveBacktrackingWithHeuristics` on the other hand can foresee problems and solves the problem with a single pass.

## Sudoku!

Coloring Australia is really a bit boring. Lets deal with something more exciting: SUDOKU! I assume you know the rules, but if you don't, go look them up, spend some time solving a few puzzles and familiarize yourself with the puzzle.
The cell below defines a string `puzzle` which represents a Sudoku puzzle from the picture below. A digit in `puzzle` corresponds to a digit in the puzzle, while an underscore `_` corresponds to a blank cell in the puzzle. For your convenience there's also variable `solution` containing the correct solution to this particular puzzle.

![](aima-images/fig_6_4.png)

In [13]:
puzzle = '''
__3_2_6__
9__3_5__1
__18_64__
__81_29__
7_______8
__67_82__
__26_95__
8__2_3__9
__5_1_3__
'''

In [14]:
solution = '''
483921657
967345821
251876493
548132976
729564138
136798245
372689514
814253769
695417382
'''

## Task 3: Sudoku as CSP

Using the string `puzzle` defined above write in the cell below code that will add all the necessary constraints to the CSP represented by the variable `sudoku`. If you can and want, write a parser that will construct the CSP from a string representation of the puzzle. Otherwise, you can construct the problem manually.

In [15]:
sudoku = CSP()
puzzle_matrix = "".join(list(filter(lambda c: c != '\n', puzzle)))
puzzle_matrix = [
    [
        puzzle_matrix[i*9 + j]
        for j in range(9)
    ]
    for i in range(9) 
]

# add all variables
for i in range(9):
    for j in range(9):
        if puzzle_matrix[i][j] == '_':
            sudoku.addVariable((i, j), {})  


for i in range(9):
    for j in range(9):
        if puzzle_matrix[i][j] != "_":
            continue

        # exclude all the options violating rules
        value_set = set([str(k) for k in range(1, 10)])

        # remove horizontal line collisions
        for k in range(9):
            if k == j:
                continue
            
            if puzzle_matrix[i][k] in value_set:
                value_set.remove(puzzle_matrix[i][k])

        # remove vertical line collisions
        for k in range(9):
            if k == i:
                continue

            if puzzle_matrix[k][j] in value_set:
                value_set.remove(puzzle_matrix[k][j])
        
        i_square = i // 3
        j_square = j // 3

        # remove smaller square collisions
        for p in range(3):
            for q in range(3):
                if (i, j) == (i_square*3 + p, j_square*3 + q):
                    continue

                if puzzle_matrix[i_square*3 + p][j_square*3 + q] in value_set:
                    value_set.remove(puzzle_matrix[i_square*3 + p][j_square*3 + q])

        sudoku.domains[(i, j)] = value_set

        # horizontal constraints
        for k in range(9):
            if k == j or puzzle_matrix[i][k] != "_":
                continue
            
            sudoku.addBinaryConstraint((i, j), "!=", (i, k))

        # vertical constraints
        for k in range(9):
            if k == i or puzzle_matrix[k][j] != "_":
                continue

            sudoku.addBinaryConstraint((i, j), "!=", (k, j))
        
        # i_square = i // 3
        # j_square = j // 3

        # smaller square constraints
        for p in range(3):
            for q in range(3):
                if (i, j) == (i_square*3 + p, j_square*3 + q) or puzzle_matrix[i_square*3 + p][j_square*3 + q] != '_':
                    continue

                sudoku.addBinaryConstraint((i, j), "!=", (i_square*3 + p, j_square*3 + q))
print(len(sudoku.domains))
                

49


Your `RecursiveBacktrackingWithHeuristics` should be able to handle this instance of sudoku with flying colors.

In [16]:
solver = RecursiveBacktrackingWithHeuristics(sudoku)
assignment = solver.solve()
print("Assignment", assignment)
print("Is consistent?", sudoku.is_consistent(assignment))
print("Is complete?", sudoku.is_complete(assignment))
print("# considered assignments", solver.counter) 

Assignment {(4, 6): '1', (4, 5): '4', (4, 2): '9', (4, 3): '5', (7, 6): '7', (6, 8): '4', (1, 6): '8', (7, 2): '4', (8, 0): '6', (7, 1): '1', (7, 7): '6', (4, 7): '3', (4, 1): '2', (4, 4): '6', (8, 8): '2', (8, 7): '8', (6, 7): '1', (3, 4): '3', (5, 4): '9', (5, 8): '5', (5, 7): '4', (3, 7): '7', (1, 7): '2', (5, 1): '3', (6, 1): '7', (8, 1): '9', (2, 1): '5', (2, 7): '9', (0, 7): '5', (3, 1): '4', (0, 1): '8', (1, 1): '6', (0, 0): '4', (0, 8): '7', (2, 0): '2', (2, 8): '3', (3, 0): '5', (3, 8): '6', (5, 0): '1', (6, 0): '3', (2, 4): '7', (1, 4): '4', (6, 4): '8', (7, 4): '5', (1, 2): '7', (0, 3): '9', (0, 5): '1', (8, 3): '4', (8, 5): '7'}
Is consistent? True
Is complete? True
# considered assignments 50


------------
The pictures are from "Artificial Intelligence: A Modern Approach" 3rd ed.